# RFM (Recency, frequency, Monetary) Segmentation 📚

## ¿Qué es la segmentación RFM?

Es un tipo de segmentación por comportamiento del cliente, basada en 3 metricas.
- **Recency (R)**: Hace cuanto fue la última transacción. Mientras menor el valor, mejor.
- **Frecuency (F)**: Cuantas compras tiene en el período analizado. Frecuencia promedio mensual también puede ser utilizada.
- **Monetary (M)**: Cuanto ha gastado el cliente en el período analizado. Monetary promedio mensual también puede ser utilizado.

Lo más común es que se realize en períodos de 12 meses, para reducir el sesgo por seasonality. Se puede acotar a diferentes períodos, depende del negocio o del ciclo de vida de los productos y/o clientes <br>
Una vez tenemos estos números, el siguiente paso es agruparlos en algún tipo de categorización como *alto, medio y bajo* <br>
Hay diferentes formas de agrupas a los clientes:
- Percentiles(quantiles,deciles,etc.): Dividir a los clientes en grupos de igual tamaño en función de los valores percentiles de cada métrica
- Pareto (80/20): Podemos asignar un valor alto al 20% de mayor puntuación y bajo al 80% de menor puntuación
- Custom: Basada en conocimiento del negocio
En la siguiente sección vamos a implementar el agrupamiento basado en percentiles.

## RFM Agrupado por percentiles.

Proceso para calcular percentiles:
1. Ordenar a los clientes basados en esa métrica
2. Dividir a los clientes en una cantidad de grupos predefinidos del mismo tamaño
3. Asignar una etiqueta a cada grupo

### Import & Read

In [1]:
import pandas as pd
import datetime

df = pd.read_excel('Datasets/Online_Retail.xlsx')
df = df.loc[~df['CustomerID'].isna()].reset_index()

# Cortamos el dataframe para que traiga un año entero
df = df.loc[df['InvoiceDate'] >= '2010-12-10']

# Calculamos la cantidad total de compra
df['TotalSum'] = df['Quantity'] * df['UnitPrice']

df.head()

,index,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalSum
14803,22523,538172,21562,HAWAIIAN GRASS SKIRT,12,2010-12-10 09:33:00,1.25,15805.0,United Kingdom,15.0
14804,22524,538172,79321,CHILLI LIGHTS,8,2010-12-10 09:33:00,4.95,15805.0,United Kingdom,39.6
14805,22525,538172,22041,"RECORD FRAME 7"" SINGLE SIZE",12,2010-12-10 09:33:00,2.55,15805.0,United Kingdom,30.6
14806,22526,538172,84558A,3D DOG PICTURE PLAYING CARDS,12,2010-12-10 09:33:00,2.95,15805.0,United Kingdom,35.4
14807,22527,538172,22952,60 CAKE CASES VINTAGE CHRISTMAS,24,2010-12-10 09:33:00,0.55,15805.0,United Kingdom,13.2


### RFM completo

In [2]:
import pandas as pd
import datetime

df = pd.read_excel('Datasets/Online_Retail.xlsx')
df = df.loc[~df['CustomerID'].isna()].reset_index()

# Cortamos el dataframe para que traiga un año entero
df = df.loc[df['InvoiceDate'] >= '2010-12-10']

# Calculamos la cantidad total de compra
df['TotalSum'] = df['Quantity'] * df['UnitPrice']

df.head()

,index,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalSum
14803,22523,538172,21562,HAWAIIAN GRASS SKIRT,12,2010-12-10 09:33:00,1.25,15805.0,United Kingdom,15.0
14804,22524,538172,79321,CHILLI LIGHTS,8,2010-12-10 09:33:00,4.95,15805.0,United Kingdom,39.6
14805,22525,538172,22041,"RECORD FRAME 7"" SINGLE SIZE",12,2010-12-10 09:33:00,2.55,15805.0,United Kingdom,30.6
14806,22526,538172,84558A,3D DOG PICTURE PLAYING CARDS,12,2010-12-10 09:33:00,2.95,15805.0,United Kingdom,35.4
14807,22527,538172,22952,60 CAKE CASES VINTAGE CHRISTMAS,24,2010-12-10 09:33:00,0.55,15805.0,United Kingdom,13.2


In [3]:
print('Min:{}; Max:{}'.format(min(df.InvoiceDate), max(df.InvoiceDate)))

Min:2010-12-10 09:33:00; Max:2011-12-09 12:50:00


#### Calcular RFM Metrics

In [4]:
# Calculamos el día que se "tomó la foto" del dataframe anterior, para calcular el recency con respecto a este día.
snapshot_date = max(df.InvoiceDate) + datetime.timedelta(days=1)

# Agregamos la data a nivel cliente
datamart = df.groupby(['CustomerID']).agg({
    'InvoiceDate':lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo':'count',
    'TotalSum':'sum'})

# Renombramos columnas para una interpretación más sencilla
datamart.rename(columns = {'InvoiceDate':'Recency',
                          'InvoiceNo':'Frequency',
                          'TotalSum': 'MonetaryValue'}, inplace = True)
datamart.head()

,Recency,Frequency,MonetaryValue
CustomerID,,,
12346.0,326,2,0.00
12347.0,2,151,3598.21
12348.0,75,31,1797.24
12349.0,19,73,1757.55
12350.0,310,17,334.40


#### Construir RFM Segments

##### Recency quartiles

La recency, a diferencia de las otras métricas, es una que mientras más pequeño sea el valor, mejor ranking tiene el cliente, por lo que usamos una lista de etiquetas inversa para hacerle justicia.

In [5]:
# Crear lista de etiquetas inversa para usar como parámetro en la siguiente línea de código
r_labels = list(range(4, 0 ,-1))

# También podemos crear etiquetas con nombres:
#r_labels = ['Active','Lapsed','Inactive','Churned']

# La función qcut ayuda a dividir en percentiles, el parámetro q nos sirve para escoger el número de grupos a dividir y en labels les ponemos etiquetas en formato lista.
r_quartiles = pd.qcut(datamart['Recency'], q=4, labels=r_labels)

# Agregamos la columna con el R score
datamart = datamart.assign(R = r_quartiles.values)

datamart

,Recency,Frequency,MonetaryValue,R
CustomerID,,,,
12346.0,326,2,0.00,1
12347.0,2,151,3598.21,4
12348.0,75,31,1797.24,2
12349.0,19,73,1757.55,3
12350.0,310,17,334.40,1
...,...,...,...,...
18280.0,278,10,180.60,1
18281.0,181,7,80.82,1
18282.0,8,13,176.60,4


#### Frequency and Monetary quartiles

Estos se calculan de forma directa (los labels se ordenan de menor a mayor). 

In [6]:
# Los labels son la única diferencia con el cálculo de Recency, todo lo demás es prácticamente igual.
f_labels = range(1,5)
m_labels = range(1,5)

f_quartiles = pd.qcut(datamart['Frequency'], q=4, labels=f_labels)
m_quartiles = pd.qcut(datamart['MonetaryValue'], q=4, labels=m_labels)

datamart = datamart.assign(F = f_quartiles.values)
datamart = datamart.assign(M = m_quartiles.values)

datamart

,Recency,Frequency,MonetaryValue,R,F,M
CustomerID,,,,,,
12346.0,326,2,0.00,1,1,1
12347.0,2,151,3598.21,4,4,4
12348.0,75,31,1797.24,2,2,4
12349.0,19,73,1757.55,3,3,4
12350.0,310,17,334.40,1,1,2
...,...,...,...,...,...,...
18280.0,278,10,180.60,1,1,1
18281.0,181,7,80.82,1,1,1
18282.0,8,13,176.60,4,1,1


##### Creamos el RFM Segment y RFM Score

El segment es basicamente concatenar como string los 3 valores RFM. <br>
El score es sumar los 3 valores. Otra forma de hacerlo es asignarle valores float entre 0 y 1 para mayor granularidad, lo cual prefiero bastante porque también facilita la ponderación de cada variable.

In [10]:
# Creamos función para concatenar los valores RFM
def join_rfm(x): return str(x['R']) + str(x['F']) + str(x['M'])

# Aplicamos la función al df y creamos una columna con el resultado.
datamart['RFM_Segment'] = datamart.apply(join_rfm, axis = 1)

# Creamos el RFM Score sumando los valores de los 3 elementos.
datamart['RFM_Score'] = datamart[['R','F','M']].sum(axis=1)
datamart.head()

,Recency,Frequency,MonetaryValue,R,F,M,RFM_Segment,RFM_Score
CustomerID,,,,,,,,
12346.0,326,2,0.00,1,1,1,111,3
12347.0,2,151,3598.21,4,4,4,444,12
12348.0,75,31,1797.24,2,2,4,224,8
12349.0,19,73,1757.55,3,3,4,334,10
12350.0,310,17,334.40,1,1,2,112,4


#### Analizando los RFM segments

In [12]:
# Ordenamos los segmentos de mayor a menor por número de clientes.
datamart.groupby('RFM_Segment').size().sort_values(ascending=False)[:10]

RFM_Segment
444    462
111    389
122    199
344    197
211    176
222    170
333    168
233    158
433    154
322    118
dtype: int64

Como podemos ver, los segmentos RFM con la calificación más baja y más alta se encuentran entre los más grandes.<br>
Siempre es la mejor práctica investigar el tamaño de los segmentos antes de usarlos para segmentación u otras aplicaciones comerciales. <br>

In [23]:
datamart.groupby('RFM_Score').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'MonetaryValue': ['mean', 'count']
}).round(1)

Recency Frequency MonetaryValue      
             mean      mean          mean count
RFM_Score                                      
3           247.5       7.9         106.3   389
4           163.3      13.9         221.3   376
5           146.7      20.8         349.7   503
6            86.8      28.1         483.7   460
7            80.2      39.2         724.8   438
8            57.4      54.8         954.0   464
9            44.0      77.2        1354.6   403
10           30.1     113.4        1795.7   443
11           19.8     190.6        3942.3   358
12            6.7     364.0        8570.6   462

Podemos ver como conforme aumenta el score, las métricas son mejores, además de ver comportamientos promedio en cada grupo para encontrar insights

Podemos agrupar a los clientes en un número menor de segmentos, utilizando el RFM Score como regla.

In [24]:
def segment_me(df):
    if df['RFM_Score'] >= 9:
        return 'Gold'
    elif (df['RFM_Score'] >= 5) and (df['RFM_Score'] < 9):
        return 'Silver'
    else:
        return 'Bronze'

datamart['General_Segment'] = datamart.apply(segment_me, axis=1)
datamart.groupby('General_Segment').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'MonetaryValue': ['mean', 'count']
}).round(1)

Recency Frequency MonetaryValue      
                   mean      mean          mean count
General_Segment                                      
Bronze            206.1      10.8         162.8   765
Gold               24.7     190.7        4029.0  1666
Silver             94.1      35.4         621.2  1865